In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["OPENAI_API_BASE"]="http://localhost:1234/v1"
os.environ["OPENAI_API_KEY"]="lm-studio"

In [ ]:
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationStore
from rich import print

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
from typing import List

In [ ]:
character = Character.generate(level=0)

In [ ]:
character

In [ ]:
region = Location.generate()
locations = LocationStore(current=region, **{region.name: region})

In [ ]:
locations

In [ ]:
city = locations.current.generate_child(type='city')

In [ ]:
city

In [ ]:
locations.add(city)
locations.set_current(city)

In [ ]:
@tool
def get_current_location():
    """\
Useful when you want to know the where you are. Returns the current location. 

Returns:
    Location: The current location.
"""
    return locations.current

@tool
def get_child_locations():
    """Useful for when you want to know child locations that exist. \
For example, the rooms in a building or the buildings in a city.

Returns:
    List[Location]: A list of child locations.
"""
    return locations.get_children(location=locations.current)

@tool
def get_parent_location():
    """\
Useful for when you want to know parent location of the current location.

Returns:
    Location: The parent location.
"""
    return locations.get_parents(location=locations.current)

@tool
def create_child_location(storyline:str=None):
    """\
Useful for when you want to create a new location that is a child of the current location. 
Use this when you want to create a city within a region, room within a building, etc. 

Args:
    storyline (str, optional): A summary of the story elements that help guide the location generation. For example, "You enter a cellar that is dark and damp."

Returns:
    Location: The new location that was created.
"""
    new_location = locations.current.generate_child(storyline)
    locations.add(new_location)
    return new_location

@tool
def create_location(location_name:str, storyline:str=None):
    """\
Useful for when you want to create a new location that is not a child of the current location. \
Returns the new location, but does not travel there.

Args:
    location_name (str): The name of the location.
    storyline (str, optional): A summary of the story elements that help guide the location generation. For example, "As you leave the village, you find a mysterious cellar."

Returns:
    Location: The new location that was created.
"""
    new_location = Location.generate(name=location_name, storyline=storyline)
    locations.add(new_location)
    return new_location

@tool
def list_locations():
    """Useful for when you want to see all locations that exist. 

Returns:
    List[str]: A list of all locations.
"""
    return list(locations.keys())


@tool
def travel_to(location_name:str):
    """\
Useful for when you want to travel to a location.

Args:
    location_name (str): The name of the location you want to travel to.
    
Returns:
    Location: The location you traveled to.
"""
    if location_name not in locations.keys():
        return "Location is not found, please create the desired location."
    locations.set_current(location)
    return location

In [ ]:
location_manager = Agent(
    role="Navigator",
    goal="Keep track of and manage new locations based on a D&D storyline.",
    backstory="""\
Driven by a sense of wonder and adventure, \
you are eager to create engaging locaitons based on a D&D story. \
You are a master of location generation and management, \
and you are always ready to create new locations and travel to them. \
""",
    tools=[
        get_current_location,
        get_child_locations,
        get_parent_location,
        create_child_location,
        create_location,
        list_locations,
        travel_to
    ]
)

In [ ]:
location_mange